In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/code/data/train/subtask_1/en/')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import transformers
import random
import re
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
df_human = pd.read_csv("human.csv")
df_bloom1b7 = pd.read_csv("bloom1b7.csv")
df_bloom3b = pd.read_csv("bloom3b.csv")
df_bloom7b1 = pd.read_csv("bloom7b1.csv")
df_gpt = pd.read_csv("gpt.csv")

In [ ]:
df_human

,id,prompt,text,label,model,domain
0,2782,NO-PROMPT,The date from which expenditure shall be eligi...,human,NO-MODEL,legal
1,3788,NO-PROMPT,Article 1. The standard import values referred...,human,NO-MODEL,legal
2,14414,NO-PROMPT,im probably more apprehensive about nm since s...,human,NO-MODEL,tweets
3,26585,NO-PROMPT,"Say something like, I will continue to look fo...",human,NO-MODEL,wiki
4,7381,NO-PROMPT,The financial support and the payments shall b...,human,NO-MODEL,legal
...,...,...,...,...,...,...
995,7725,NO-PROMPT,The date 31 December 1994 is hereby replaced b...,human,NO-MODEL,legal
996,31383,NO-PROMPT,Your weight should shift from your front (left...,human,NO-MODEL,wiki
997,19081,NO-PROMPT,Parents STILL havent some back from the hospit...,human,NO-MODEL,tweets
998,14700,NO-PROMPT,andy roddick advances to ROUND 4! hes gonna me...,human,NO-MODEL,tweets


# BLOOM-1B7

In [ ]:
def clean_generated_text(generated_text):
    cleaned_text = re.search(r'Your prediction:\n?(.+)', generated_text, re.DOTALL)
    if cleaned_text:
      response = cleaned_text.group(1).strip()
      if 'machine' in response.lower() :
          return 'generated'
      elif 'LLM' in response:
          return 'generated'
      elif 'language model' in response.lower():
          return 'generated'
      elif 'human' in response.lower():
          return 'human'
      else:
          return response
    else:
        return None

In [ ]:
prompt_bloom1b7 = """Your task is to evaluate the following text and determine whether it was written by a human or generated by a language model.
        Text: {}

Based on your analysis, please provide your judgment.
If you believe the text was generated by a Large Language Model (LLM), answer '<s>The text is written by a machine.</s>'.
If you believe the text was written by a human, answer '<s>The text is written by a human.</s>'.

Your prediction: The text is"""

In [ ]:
prompt_bloom3b = """Text classification: Your task is to evaluate the following text and determine whether it was written by a human or generated by a language model.
        Text: {}

Choose the most likely option:
If you believe the text was generated by a Large Language Model (LLM), answer '<s>The text is generated by a machine.</s>'.
If you believe the text was written by a human, answer '<s>The text is written by a human.</s>'.

Your prediction: The text is"""

In [ ]:
def classify_texts(df, output_path, model_name, prompt):


    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Initialize a list to store the results
    classification_results = []

    # Loop through each text in the dataset
    for index, row in df.iterrows():

        text_to_classify = row['text']
        text_id = row['id']
        text = row['text']
        label = row['label']
        domain = row['domain']

        prompt_formatted = prompt.format(text_to_classify)

        inputs = tokenizer(prompt_formatted, return_tensors="pt")
        input_length = inputs.input_ids.size(1)
        max_length = max(50, input_length+25)
        generated_ids = model.generate(inputs.input_ids, max_length=max_length, temperature = 0.5, do_sample = True) # change temperature accordingly
        generated_text = tokenizer.decode(generated_ids[0])

        cleaned_generated_text = clean_generated_text(generated_text)

        classification_results.append({'id': text_id, 'text': text, 'domain': domain, 'label' : label,  'predicted_label': cleaned_generated_text})

    # Convert the results to a DataFrame
    print(classification_results)
    classification_df = pd.DataFrame(classification_results)

    # Save the DataFrame to a CSV file
    classification_df.to_csv(output_path, index=False)

In [4]:
def results_evaluation(file_path, correct_answer):
    df = pd.read_csv(file_path)

    df['predicted_label'] = df['predicted_label'].where(df['predicted_label'].isin(['generated', 'human']))

    df.dropna(subset=['predicted_label'], inplace=True)

    df['correct_answer'] = correct_answer

    report = classification_report(df['correct_answer'], df['predicted_label'], zero_division=1)

    micro_f1 = f1_score(df['correct_answer'], df['predicted_label'], average='micro')

    print(df['predicted_label'].value_counts())

    return micro_f1, report

## Human texts classification

In [ ]:
classify_texts(df_human, "classification_results_1b7_human-2.csv", "bigscience/bloom-1b7", prompt_bloom1b7)

In [5]:
report, f1 = results_evaluation("classification_results_1b7_human.csv", "human" )
print(report)

print(f1)

#print(matrix)

predicted_label
generated    623
human        376
Name: count, dtype: int64
0.3763763763763764
              precision    recall  f1-score   support

   generated       0.00      1.00      0.00         0
       human       1.00      0.38      0.55       999

    accuracy                           0.38       999
   macro avg       0.50      0.69      0.27       999
weighted avg       1.00      0.38      0.55       999



In [ ]:
classify_texts(df_human, "classification_results_3b_human.csv", "bigscience/bloom-3b", prompt_bloom3b)

In [ ]:
report, f1 = results_evaluation("classification_results_3b_human.csv", "human" )
print(report)

print(f1)

predicted_label
human        578
generated    421
Name: count, dtype: int64
0.5785785785785785
              precision    recall  f1-score   support

   generated       0.00      1.00      0.00         0
       human       1.00      0.58      0.73       999

    accuracy                           0.58       999
   macro avg       0.50      0.79      0.37       999
weighted avg       1.00      0.58      0.73       999



## BLOOM-1B7 texts classification

In [ ]:
classify_texts(df_bloom1b7, "classification_results_1b7_1b7.csv", "bigscience/bloom-1b7", prompt_bloom_1b7)

In [6]:
report, f1 = results_evaluation("classification_results_1b7_1b7.csv", "generated" )
print(report)

print(f1)

predicted_label
generated    639
human        361
Name: count, dtype: int64
0.639
              precision    recall  f1-score   support

   generated       1.00      0.64      0.78      1000
       human       0.00      1.00      0.00         0

    accuracy                           0.64      1000
   macro avg       0.50      0.82      0.39      1000
weighted avg       1.00      0.64      0.78      1000



In [ ]:
classify_texts(df_bloom1b7, "classification_results_3b_1b7.csv", "bigscience/bloom-3b")

In [7]:
report, f1 = results_evaluation("classification_results_3b_1b7.csv", "generated" )
print(report)

print(f1)

predicted_label
human        551
generated    449
Name: count, dtype: int64
0.449
              precision    recall  f1-score   support

   generated       1.00      0.45      0.62      1000
       human       0.00      1.00      0.00         0

    accuracy                           0.45      1000
   macro avg       0.50      0.72      0.31      1000
weighted avg       1.00      0.45      0.62      1000



## BLOOM-3B texts classification

In [ ]:
classify_texts(df_bloom3b, "classification_results_1b7_3b.csv", "bigscience/bloom-1b7", prompt_bloom_1b7)

In [8]:
report, f1 = results_evaluation("classification_results_1b7_3b.csv", "generated" )
print(report)

print(f1)

predicted_label
generated    609
human        390
Name: count, dtype: int64
0.6096096096096096
              precision    recall  f1-score   support

   generated       1.00      0.61      0.76       999
       human       0.00      1.00      0.00         0

    accuracy                           0.61       999
   macro avg       0.50      0.80      0.38       999
weighted avg       1.00      0.61      0.76       999



In [ ]:
classify_texts(df_bloom3b, "classification_results_3b_3b.csv", "bigscience/bloom-3b", prompt_bloom3b)

In [ ]:
report, f1 = results_evaluation("classification_results_3b_3b.csv", "generated" )
print(report)

print(f1)

predicted_label
human        571
generated    429
Name: count, dtype: int64
0.429
              precision    recall  f1-score   support

   generated       1.00      0.43      0.60      1000
       human       0.00      1.00      0.00         0

    accuracy                           0.43      1000
   macro avg       0.50      0.71      0.30      1000
weighted avg       1.00      0.43      0.60      1000



## GPT texts classification

In [ ]:
classify_texts(df_gpt, "classification_results_1b7_gpt.csv", "bigscience/bloom-1b7", prompt_bloom1b7)

In [9]:
report, f1 = results_evaluation("classification_results_1b7_gpt.csv", "generated" )
print(report)

print(f1)

predicted_label
generated    600
human        399
Name: count, dtype: int64
0.6006006006006006
              precision    recall  f1-score   support

   generated       1.00      0.60      0.75       999
       human       0.00      1.00      0.00         0

    accuracy                           0.60       999
   macro avg       0.50      0.80      0.38       999
weighted avg       1.00      0.60      0.75       999



In [ ]:
classify_texts(df_gpt, "classification_results_3b_gpt.csv", "bigscience/bloom-1b7", prompt_bloom1b7)

In [10]:
report, f1 = results_evaluation("classification_results_3b_gpt.csv", "generated" )
print(report)

print(f1)

predicted_label
generated    507
human        493
Name: count, dtype: int64
0.507
              precision    recall  f1-score   support

   generated       1.00      0.51      0.67      1000
       human       0.00      1.00      0.00         0

    accuracy                           0.51      1000
   macro avg       0.50      0.75      0.34      1000
weighted avg       1.00      0.51      0.67      1000

